# Loop 12 Analysis: Strategic Pivot Required

## Key Observations:
1. 12 experiments all converge to ~70.66 (local optimum)
2. Target is 68.92 - 1.74 points below current best
3. Local optimization (SA, bbox3) provides negligible improvement (0.000001)
4. Need fundamentally different approach

In [ ]:
import pandas as pd
import numpy as np
import os
import json

# Load session state to understand experiment history
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

print(f"Total experiments: {len(state['experiments'])}")
print(f"\nExperiment scores:")
for exp in state['experiments']:
    print(f"  {exp['id']}: {exp['name'][:50]:50s} | Score: {exp['score']:.6f}")

print(f"\nBest score: {min(e['score'] for e in state['experiments']):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {min(e['score'] for e in state['experiments']) - 68.919154:.6f}")

In [ ]:
# Analyze what approaches have been tried
approaches_tried = [
    ('Pre-optimized baseline', 70.659959, 'No improvement possible'),
    ('C++ SA optimizer', 70.659959, '0 improvement'),
    ('Lattice construction', 88.33, 'Much worse'),
    ('Lattice + SA', 85.93, 'Still much worse'),
    ('Eazy optimizer', 70.659944, '0.000015 improvement'),
    ('Rotation optimization', 70.659959, 'Negative improvement'),
    ('bbox3 short runs', 70.659958666, '0.000001 improvement'),
    ('Tree removal', 70.659959, '0 improvement'),
    ('bbox3 with repair', 70.659958437, '0.000001 improvement'),
    ('bbox3 multi-phase', 70.659958593, '0.000001 improvement'),
]

print("Approaches tried and results:")
print("="*80)
for name, score, result in approaches_tried:
    print(f"{name:30s} | Score: {score:.6f} | {result}")

print("\n" + "="*80)
print("CONCLUSION: All local optimization approaches converge to ~70.66")
print("The baseline is at an EXTREMELY strong local optimum")
print("Need GLOBAL search or fundamentally different representation")

In [ ]:
# What approaches have NOT been tried?
approaches_not_tried = [
    'Genetic Algorithm with crossover',
    'Basin hopping (random perturbation + local opt)',
    'Constraint Programming (CP-SAT)',
    'Different starting points (not saspav)',
    'Asymmetric solutions (mentioned in discussions)',
    'Focus on small N values (N=1-20 have worst efficiency)',
    'Hexagonal/triangular lattice with different parameters',
    'Greedy constructive with different orderings',
    'Backward iteration (from N=200 down)',
]

print("Approaches NOT yet tried:")
print("="*80)
for i, approach in enumerate(approaches_not_tried, 1):
    print(f"{i}. {approach}")

print("\n" + "="*80)
print("PRIORITY: Try approaches that can ESCAPE the local optimum")

In [ ]:
# Check what's in the snapshots - maybe there are better solutions
import glob

snapshot_dirs = glob.glob('/home/nonroot/snapshots/santa-2025/*/')
print(f"Found {len(snapshot_dirs)} snapshot directories")

# Check the most recent ones for submission files
for snap_dir in sorted(snapshot_dirs)[-5:]:
    sub_path = os.path.join(snap_dir, 'submission', 'submission.csv')
    if os.path.exists(sub_path):
        print(f"\n{snap_dir}:")
        df = pd.read_csv(sub_path)
        print(f"  Rows: {len(df)}")
        # Check first few rows
        print(df.head(3))

In [ ]:
# Let's look at the current best submission and understand its structure
baseline_path = '/home/code/external_data/saspav/santa-2025.csv'
df = pd.read_csv(baseline_path)

print(f"Baseline submission shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst 10 rows:")
print(df.head(10))

# Extract N values
df['N'] = df['id'].str.split('_').str[0].astype(int)
print(f"\nN values range: {df['N'].min()} to {df['N'].max()}")
print(f"Total configurations: {df['N'].nunique()}")

In [ ]:
# Analyze per-N scores to find where improvements are most needed
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def get_score_for_n(df, n):
    group = df[df['N'] == n]
    trees = []
    for _, row in group.iterrows():
        x = str(row['x']).lstrip('s')
        y = str(row['y']).lstrip('s')
        deg = str(row['deg']).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    
    if not trees:
        return float('inf')
    
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    
    width = maxx - minx
    height = maxy - miny
    side_length = max(width, height)
    
    return float(side_length * side_length / n)

print("Calculating per-N scores (this may take a minute)...")
per_n_scores = {}
for n in range(1, 201):
    per_n_scores[n] = get_score_for_n(df, n)
    if n % 20 == 0:
        print(f"  N={n}: {per_n_scores[n]:.6f}")

print(f"\nTotal score: {sum(per_n_scores.values()):.6f}")

In [ ]:
# Find N values with worst efficiency (highest score contribution)
import matplotlib.pyplot as plt

scores_df = pd.DataFrame([
    {'N': n, 'score': s, 'efficiency': s * n / (n * 1.0)}  # s^2/n, so efficiency = s^2
    for n, s in per_n_scores.items()
])

# Sort by score (highest first = worst efficiency)
worst_n = scores_df.nlargest(20, 'score')
print("Top 20 N values with WORST efficiency (highest score contribution):")
print(worst_n.to_string(index=False))

# Calculate cumulative contribution
scores_df = scores_df.sort_values('score', ascending=False)
scores_df['cumulative_pct'] = scores_df['score'].cumsum() / scores_df['score'].sum() * 100
print(f"\nTop 10 N values contribute {scores_df.head(10)['score'].sum() / scores_df['score'].sum() * 100:.1f}% of total score")

In [ ]:
# Key insight: What would it take to reach the target?
target = 68.919154
current = sum(per_n_scores.values())
gap = current - target

print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap to close: {gap:.6f}")
print(f"Gap percentage: {gap/current*100:.2f}%")

# If we improved each N proportionally, how much improvement per N?
avg_improvement_per_n = gap / 200
print(f"\nAverage improvement needed per N: {avg_improvement_per_n:.6f}")

# Focus on worst N values - if we improved just the top 20 worst
top20_contribution = scores_df.head(20)['score'].sum()
print(f"\nTop 20 worst N values contribute: {top20_contribution:.6f} ({top20_contribution/current*100:.1f}%)")
print(f"If we improved these by 10%: {top20_contribution * 0.1:.6f} improvement")
print(f"If we improved these by 20%: {top20_contribution * 0.2:.6f} improvement")

In [ ]:
# Summary and recommendations
print("="*80)
print("STRATEGIC ANALYSIS SUMMARY")
print("="*80)

print("""
1. CURRENT STATE:
   - Best score: 70.659959
   - Target: 68.919154
   - Gap: 1.74 points (2.5%)
   - All local optimization approaches converge to ~70.66

2. WHY LOCAL OPTIMIZATION FAILS:
   - The saspav baseline is at an EXTREMELY strong local optimum
   - SA, bbox3, rotation optimization all show ~0 improvement
   - The solution space has many local optima, but we're stuck in one

3. APPROACHES TO TRY:
   a) GENETIC ALGORITHM:
      - Crossover between different configurations
      - Can escape local optima by combining good parts
   
   b) BASIN HOPPING:
      - Random large perturbation + local optimization
      - Explores different basins of attraction
   
   c) FOCUS ON SMALL N:
      - N=1-20 have worst efficiency
      - Manual optimization or exhaustive search may be feasible
   
   d) DIFFERENT STARTING POINTS:
      - Don't start from saspav baseline
      - Try lattice-based or random initial solutions
   
   e) ASYMMETRIC SOLUTIONS:
      - Discussions mention asymmetric solutions outperform symmetric
      - Break symmetry in current solutions

4. PRIORITY:
   - Try Genetic Algorithm first (can escape local optima)
   - Focus on small N values (highest improvement potential)
   - Explore different starting configurations
""")

print("="*80)